In [3]:
# !wget https://raw.githubusercontent.com/jla-gardner/graph-pes/refs/heads/main/scripts/build-lammps.sh
# !bash build-lammps.sh --help
!bash ../../../scripts/build-lammps.sh --help

Usage: ../../../scripts/build-lammps.sh [OPTIONS]

This script builds a LAMMPS executable with support for 'pair_style graph_pes'.

It performs the following tasks:
  1. Locates the graph-pes installation
  2. Creates a conda environment with necessary dependencies
  3. Clones the LAMMPS repository
  4. Patches LAMMPS with graph-pes source code
  5. Builds LAMMPS with graph-pes support

The final executable will be located at:
  ./graph_pes_lmp_cpu_only (if --cpu-only is used)
  ./graph_pes_lmp (default, with GPU support)

Requirements:
  - conda

Options:
  --help           Display this help message and exit
  --cpu-only       Build LAMMPS for CPU only (default: GPU enabled)
  --force-rebuild  Force rebuilding of conda environment and LAMMPS



In [4]:
!bash build-lammps.sh

Running build-lammps.sh with the following parameters:
          CPU_ONLY: false
    FORCE_REBUILD : false
Found graph-pes pair style at (...)/graph_pes/pair_style
Creating conda environment lammps-env-gpu-throwaway
Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: ...working... done

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
(...)

done
#
# To activate this environment, use
#
#     $ conda activate lammps-env-gpu-throwaway
#
# To deactivate an active environment, use
#
#     $ conda deactivate

Conda environment lammps-env-gpu-throwaway successfully activated
Cloning into 'lammps'...
remote: Enumerating objects: 403276, done.
remote: Counting objects: 100% (3486/3486), done.
remote: Compressing objects: 100% (1504/1504), done.
remote: Total 403276 (delta 2317), reused 3036 (delta 1977), pack-reused 399790 (from 1)
Receiving objects: 100% (403276/403276), 757.23 MiB | 52.42 Mi

We can check that our ``graph_pes_lmp`` executable works by inspecting the list of available ``pair_style`` modules:

In [6]:
!./graph_pes_lmp -h | grep -A8 "* Pair styles:"

* Pair styles:

born            buck            buck/coul/cut   coul/cut        coul/debye      
coul/dsf        coul/wolf       meam/c          reax            reax/c          
mesont/tpm      graph_pes       hybrid          hybrid/omp      hybrid/molecular                
hybrid/molecular/omp            hybrid/overlay  hybrid/overlay/omp              
hybrid/scaled   hybrid/scaled/omp               lj/cut          lj/cut/coul/cut 
lj/expand       morse           soft            table           yukawa          
zbl             zero 


In [21]:
from graph_pes.deploy import deploy_model
from graph_pes.models import LennardJones

model = LennardJones(epsilon=0.4, sigma=1.2, cutoff=5.0)
deploy_model(model, "lj_model_lammps.pt")

In [22]:
!ls | grep ".*pt"

lj_model_lammps.pt


In [15]:
from ase.build import bulk

atoms = bulk("Cu", "hcp", a=3.6, c=5.8)
atoms = atoms.repeat(3)
atoms.write("starting-structure.data", format="lammps-data")

In [20]:
!cat starting-structure.data | head -n 15

starting-structure.data (written by ASE) 

54 	 atoms 
1  atom types
0.0      10.800000000000001  xlo xhi
0.0      9.3530743608719362  ylo yhi
0.0      17.399999999999999  zlo zhi
    -5.4000000000000004                       0                       0  xy xz yz


Atoms 

     1   1                       0                       0                       0
     2   1 -7.4014868308343753e-18      2.0784609690826525      2.8999999999999999
     3   1                       0                       0      5.7999999999999998


In [23]:


input_script = """
# a basic NVT simulation using pair_style graph_pes
# 
# expected variables:
#   output_dir: path to output directory
#   temp_K: target temperature in K
#   model_file: path to model file
#   atom_types: list of element symbols
#   data_file: path to input data file


log ${output_dir}/log.lammps

# Set units to 'real' for atomic units (fs, kcal/mol, etc.)
units metal
atom_style atomic
newton off

# Read initial structure from data file
read_data ${data_file}

# Define graph-pes pair style
pair_style      graph_pes
pair_coeff      * * ${model_file} ${atom_types}

# Define neighbor list
neighbor 0.5 bin
neigh_modify delay 0 every 1 check yes

# timestep of 1 fs
timestep 0.001

# Setup NVT ensemble with Nose-Hoover thermostat
# relaxation time of 10 fs
fix 1 all nvt temp ${temp_K} ${temp_K} 0.01

# Dump output every 50 timesteps
dump 1 all atom 50 ${output_dir}/dumps/*.data

# Thermodynamic output every 100 steps
thermo 100
thermo_style custom step time temp etotal press

# Run for 10 ps
run 10000
"""

!echo "${input_script}" > lammps_input.in

In [ ]:
%% bash

mkdir -p nvt-simulation/dumps
./graph_pes_lmp -in lammps_input.in \
  output_dir="nvt-simulation" \
  temp_K=400 \
  model_file="lj_model_lammps.pt" \
  atom_types="Cu" \
  data_file="starting-structure.data"